# *Dataframe* operations
Before proceeding with the Water Flow exercise, let's first pause and discuss data structures: **vectors**/**series**, **matrices**, **N-dimensional arrays**, and particularly the **dataframe**.

## What is a dataframe?
The **dataframe** is a key tool in data analysis. Dataframes store data a specific format, one that facilitates many different types of analyses. This format is a table similar to an Excel worksheet, but has more strict conventions:
* Each **<u>column</u>** represents a **field**. It has a header or label and the values it holds all have the same **data type**.
* Each **<u>row</u>** represents an **observation**. All values in a given row related in that they describe the same entity. 
* Rows are typically referenced by an **<u>index</u>**. Index values are often, but not always, non-repeating sequential integer values.

## Typical operations in a dataframe
Once data are organized in a dataframe, it's quite straightforward to do the following:
* <u>Select</u>/<u>filter</u>/<u>sort</u> data by row, by column, or by both. 
* <u>Compute</u> new fields from existing ones
* <u>Combine</u> tables, either by appending columns or rows 
* <u>Reshape</u> tables either by melting, pivoting
* <u>Summarizing</u>/<u>Grouping</u> data
* Handling <u>missing data</u> 
* <u>Plotting</u>  data

We'll introduce how a few of these operations are done using the Python `Pandas` here in this notebook. Specifically, we'll examine how to subset rows and columns can be selected from dataframes as this offers more insight on how dataframes are organized and manipulated in Python. The other tasks will be examined in subsquent notebooks. 

## Diving in...
We'll begin by importing Pandas and then loading the water flow dataset retreived in the previous notebook. 

In [1]:
#import libraries
import pandas as pd

In [2]:
#Load data from the server into a dataframe named 'df'
url = 'http://waterservices.usgs.gov/nwis/dv/?format=rdb&sites=02087500&startDT=1930-10-01&endDT=2017-09-30&statCd=00003&parameterCd=00060&siteStatus=all'
df = pd.read_csv(url,
                 skiprows=31,
                 sep='\t',
                 names=['agency_cd','site_no','datetime','MeanFlow_cfs','Confidence'],
                 dtype={'site_no':'str'},
                 parse_dates=['datetime']
                )

In [3]:
#Display each column's data type
df.dtypes

agency_cd               object
site_no                 object
datetime        datetime64[ns]
MeanFlow_cfs           float64
Confidence              object
dtype: object

## Selecting data
Selecting data, aka "filtering", "subsetting", "slicing", etc., can be done by column, by row, or both. Identifying the rows or columns can be done by position, index/label, or by query, as we'll see in the following examples. 

### Selecting specific *columns* of data
Isolating a specific column of data is fairly straightforward; we just enter, in brackets the name of the column.

In [4]:
#Create a new array from just the one column
dfMeanFlow = df['MeanFlow_cfs']

In [5]:
#Show the first 5 rows of that array
dfMeanFlow.head()

0    173.0
1    132.0
2    125.0
3    125.0
4     98.0
Name: MeanFlow_cfs, dtype: float64

To isolate more than one column, we just pass a *list* of column names between the brackets.<br>*Note that lists in Python are themselves surrounded by brackets `[]`.*

In [6]:
#Create a new dataframe from the Flow and Confidence columns
df2 = df[['MeanFlow_cfs','Confidence']]
df2.head()

,MeanFlow_cfs,Confidence
0,173.0,A
1,132.0,A
2,125.0,A
3,125.0,A
4,98.0,A


### Selecting rows of data...
There are a few means for selecting rows of data: by position, by index, or query, or by mask... Here, we'll touch on each.

### ♦ Selecting by position with `iloc`
First, we can pull one or a **slice** of rows by the row's sequential position in the dataframe using the `iloc` command (short for **i**nteger **loc**ation).<br>*Recall that in Python, lists begin at zero, not one...*

* Single values

In [7]:
#Show first row
df.iloc[0]

agency_cd                      USGS
site_no                    02087500
datetime        1930-10-02 00:00:00
MeanFlow_cfs                    173
Confidence                        A
Name: 0, dtype: object

In [8]:
#Show second row
df.iloc[1]

agency_cd                      USGS
site_no                    02087500
datetime        1930-10-03 00:00:00
MeanFlow_cfs                    132
Confidence                        A
Name: 1, dtype: object

In [9]:
#Show last row
df.iloc[-1]

agency_cd                      USGS
site_no                    02087500
datetime        2017-09-30 00:00:00
MeanFlow_cfs                    191
Confidence                        A
Name: 31775, dtype: object

<font color=red>► What would the command be to show the 100th row of data?</font>

In [ ]:
#Show 100th row
df.iloc[]

* Data *slices*: A slice of data is a set of contiguous rows (or columns). We can slice our data with `iloc` by providing the bounds of the slice we want.<br>*Note that the upper bound is not included in the slice*. 

In [10]:
#Show the first 4 rows. 
df.iloc[0:4]

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence
0,USGS,02087500,1930-10-02,173.0,A
1,USGS,02087500,1930-10-03,132.0,A
2,USGS,02087500,1930-10-04,125.0,A
3,USGS,02087500,1930-10-05,125.0,A


In [11]:
#Show the first 4 rows (again): Note that if we omit the lower bound, it assumes it's zero
df.iloc[:4]

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence
0,USGS,02087500,1930-10-02,173.0,A
1,USGS,02087500,1930-10-03,132.0,A
2,USGS,02087500,1930-10-04,125.0,A
3,USGS,02087500,1930-10-05,125.0,A


In [12]:
#Show rows 100 thru 105
df.iloc[100:106]

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence
100,USGS,02087500,1931-01-10,645.0,A
101,USGS,02087500,1931-01-11,595.0,A
102,USGS,02087500,1931-01-12,750.0,A
103,USGS,02087500,1931-01-13,1680.0,A
104,USGS,02087500,1931-01-14,1610.0,A
105,USGS,02087500,1931-01-15,1350.0,A


<font color=red>► What would the command be to show the last 5 records?</font>

In [ ]:
#Show the last 5 rows 
df.iloc[]

* Selecting rows *and columns* using `iloc`. 
Since tables are 2 dimensional, we can easily select/slice data by column or row AND column with `iloc.

In [13]:
#First, remind us what our columns are
df.columns

Index(['agency_cd', 'site_no', 'datetime', 'MeanFlow_cfs', 'Confidence'], dtype='object')

In [14]:
#Select the flow data (4th column) for the 100th record
df.iloc[99,3]

860.0

In [15]:
#Select the flow data and confidence values for the 100th to 110th records
df.iloc[99:110,3:]

,MeanFlow_cfs,Confidence
99,860.0,A
100,645.0,A
101,595.0,A
102,750.0,A
103,1680.0,A
104,1610.0,A
105,1350.0,A
106,980.0,A
107,860.0,A
108,860.0,A


---
### ♦ Selecting by index with `loc`
While `iloc` references rows by their actual position in the data frame, `loc` references them by their **index**. Let's first examine this using the auto-generated indices created when we imported the CSV into a dataframe. Running the `index` function reveals that our initial index was assigned a sequential range of integers. 

In [16]:
#What does our index look like? 
df.index

RangeIndex(start=0, stop=31776, step=1)

In [17]:
#Show the rows corresponding to index values 6 thru 10
df.loc[6:10]

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence
6,USGS,02087500,1930-10-08,87.0,A
7,USGS,02087500,1930-10-09,118.0,A
8,USGS,02087500,1930-10-10,118.0,A
9,USGS,02087500,1930-10-11,118.0,A
10,USGS,02087500,1930-10-12,111.0,A


Now, let's change our index from the autogenerated sequential values to the values in stored in the `datetime` column.

In [18]:
#Change the index to be values in the datetime column and display them
df.set_index('datetime',inplace=True)
df.index

DatetimeIndex(['1930-10-02', '1930-10-03', '1930-10-04', '1930-10-05',
               '1930-10-06', '1930-10-07', '1930-10-08', '1930-10-09',
               '1930-10-10', '1930-10-11',
               ...
               '2017-09-21', '2017-09-22', '2017-09-23', '2017-09-24',
               '2017-09-25', '2017-09-26', '2017-09-27', '2017-09-28',
               '2017-09-29', '2017-09-30'],
              dtype='datetime64[ns]', name='datetime', length=31776, freq=None)

In [19]:
#Show the row with the index matching Jan 1st, 1975
df.loc['1975-01-01']

agency_cd           USGS
site_no         02087500
MeanFlow_cfs        1170
Confidence             A
Name: 1975-01-01 00:00:00, dtype: object

In [20]:
#Show the slice of rows spanning september 10th thru 15th, 1998
df.loc['1998-09-10':'1998-09-15']

,agency_cd,site_no,MeanFlow_cfs,Confidence
datetime,,,,
1998-09-10,USGS,02087500,364.0,A
1998-09-11,USGS,02087500,334.0,A
1998-09-12,USGS,02087500,308.0,A
1998-09-13,USGS,02087500,298.0,A
1998-09-14,USGS,02087500,288.0,A
1998-09-15,USGS,02087500,285.0,A


In [21]:
#Return select rows AND columns using loc
df.loc['1998-09-10':'1998-09-15','MeanFlow_cfs':'Confidence']

,MeanFlow_cfs,Confidence
datetime,,
1998-09-10,364.0,A
1998-09-11,334.0,A
1998-09-12,308.0,A
1998-09-13,298.0,A
1998-09-14,288.0,A
1998-09-15,285.0,A


<font color=red>► Use `loc` to return `MeanFlow_cfs` data for Sept 1, 2017</font>

In [ ]:
df.loc[]

<font color=red>► Use `loc` to return `MeanFlow_cfs` data Sept, 2017 onward to the end of the dataset</font>

In [ ]:
df.loc[]

### ♦ Selecting by querying data
Moving away from indices, we can query records matching criteria that we specify.  

In [22]:
#Select rows where the Mean flow was less than 50 cfs
df.query('MeanFlow_cfs < 50')

,agency_cd,site_no,MeanFlow_cfs,Confidence
datetime,,,,
1932-09-11,USGS,02087500,45.0,A
1932-09-15,USGS,02087500,48.0,A
1932-09-20,USGS,02087500,46.0,A
1933-10-08,USGS,02087500,49.0,A
1933-10-09,USGS,02087500,48.0,A


In [23]:
#Select rows where the Confidence indicates estimated:
df.query('Confidence == "A:e"')

,agency_cd,site_no,MeanFlow_cfs,Confidence
datetime,,,,
1986-07-10,USGS,02087500,300.0,A:e
1986-07-11,USGS,02087500,320.0,A:e
1986-07-12,USGS,02087500,300.0,A:e
1986-07-13,USGS,02087500,305.0,A:e
1986-07-14,USGS,02087500,310.0,A:e
1987-05-14,USGS,02087500,600.0,A:e
1988-10-14,USGS,02087500,210.0,A:e
1988-10-15,USGS,02087500,220.0,A:e
1988-10-16,USGS,02087500,230.0,A:e


<font color=red>► Query the data for mean flow values equal to 55</font>

In [ ]:
df.query()

## ♦ Using *masks* to query data
This method is a bit more convoluted. First we create a **mask** which is a binary column of data, meaning values are either true or false, by supplying a criteria. And then we **apply the mask**, which returns only those records that are true. 

In [24]:
#Create a mask of flows below 53 cfs
maskTinyFlow = df['MeanFlow_cfs'] < 53

In [25]:
#Apply the mask; this will only return rows where the mask was true
df[maskTinyFlow]

,agency_cd,site_no,MeanFlow_cfs,Confidence
datetime,,,,
1932-09-11,USGS,02087500,45.0,A
1932-09-12,USGS,02087500,50.0,A
1932-09-15,USGS,02087500,48.0,A
1932-09-20,USGS,02087500,46.0,A
1933-10-06,USGS,02087500,52.0,A
1933-10-07,USGS,02087500,52.0,A
1933-10-08,USGS,02087500,49.0,A
1933-10-09,USGS,02087500,48.0,A
1933-10-10,USGS,02087500,51.0,A


## Recap
Clearly, we are just scratching the surface of what we can do when our data is in a dataframe. However, in the next few notebooks, we'll dig a bit deeper by re-examining our water flow exercise. 